## 1. 각 클러스터 parameter 저장 및 scaling

In [1]:
from scipy.stats import dirichlet
import numpy as np

In [2]:
# 클러스터의 parameter
fields = ['기술과학', '문학', '사회과학', '역사', '예술', '자연과학', '종교', '철학']

cluster1 = [1.579,1.817,1.877,2.736,1.602, 1.7, 2.891, 1.567] # 석가모니 클러스터
cluster2 = [1.926, 1.983, 2.65, 1.499, 2.561, 1.129, 1.456, 2.87] # 소크라테스 클러스터
cluster3 = [2.488, 1.915, 1.903, 1.613, 2.019, 2.769, 1 ,1.957] # 맥가이버 클러스터

In [3]:
# scaling 함수 (리스트 값 합이 x가 되도록 함)

def scale_list(my_list, k):
    total_sum = sum(my_list)
    scaling_factor = k / total_sum
    scaled_list = [x * scaling_factor for x in my_list]
    return scaled_list

In [4]:
# scaling 진행

cluster1 = scale_list(cluster1, 16)
cluster2 = scale_list(cluster2, 16)
cluster3 = scale_list(cluster3, 16)

In [6]:
# 영상 분야 추출 example (2개의 영상을 입력받았다고 가정)

obs1 = [0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.99, 0.0012, 0.0012]
obs2 = [0.013, 0.013, 0.013, 0.137, 0.013, 0.595, 0.137, 0.013]

In [7]:
# scaling 진행

obs1 = scale_list(obs1, 1)
obs2 = scale_list(obs2, 1)

## 2. likelihood 계산 및 클러스터 할당

In [8]:
# 디리클레 분포 객체 생성
dist1 = dirichlet(cluster1)
dist2 = dirichlet(cluster2)
dist3 = dirichlet(cluster3)

# 가능도 계산
cluster1like = dist1.pdf(obs1)*dist1.pdf(obs2)
cluster2like = dist2.pdf(obs1)*dist2.pdf(obs2)
cluster3like = dist3.pdf(obs1)*dist3.pdf(obs2)

In [9]:
# 클러스터 할당

if max(cluster1like, cluster2like, cluster3like) == cluster1like:
    print("석가모니")
    prior = cluster1

elif max(cluster1like, cluster2like, cluster3like) == cluster2like:
    print("소크라테스")
    prior = cluster2

else:
    print("맥가이버")
    prior = cluster3

맥가이버


## 3. Posterior Dist. (사후분포) 생성

In [10]:
posterior = np.sum([prior, obs1, obs2], axis=0)
posterior

array([2.5564893 , 1.97119818, 1.95894078, 1.79548246, 2.07742903,
       4.45702783, 1.16933332, 2.0140991 ])

## 4. 분야 Sampling

In [11]:
# 디리클레분포에서 선호 분포 parameter 형성
prefer_param = np.random.dirichlet(posterior)
prefer_param

# 다항분포에서 샘플링
sample = np.random.multinomial(1, prefer_param)

# 샘플링된 분야 출력
selected_field = fields[np.argmax(sample)]

print(selected_field)

역사


### ※ (참고)

In [12]:
# 다항분포에서 1000번 샘플링
samples = np.random.multinomial(1000, prefer_param, size=1)[0]

# 각 분야의 빈도 계산
frequencies = dict(zip(fields, samples))

# 결과 출력
for field, count in frequencies.items():
    print(f'{field}: {count}')

기술과학: 138
문학: 134
사회과학: 213
역사: 23
예술: 85
자연과학: 193
종교: 81
철학: 133


* 맥가이버 클러스터로 할당된 사용자의 경우, 자연과학과 기술과학의 분야가 많이 샘플링되고 있음을 확인 가능함.    
또, 가장 높은 값을 배정해주는 것이 아닌 샘플링을 통해 추천이 이루어지기 때문에 사용자는 다양한 분야의 책을 접할 수 있음